In [4]:
import os
import sys
import time
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from detection_estimation.segment_mapping import *
import matplotlib.pyplot as plt
from apperception.utils import fetch_camera_config, fetch_camera_trajectory

In [5]:
# test_config = fetch_camera_config('samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385108912404.jpg', database)
# test_config

In [6]:
# mapping = map_imgsegment_roadsegment(test_config)

In [7]:
# test_file_path = '/home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385108912404.jpg'
# visualization(test_file_path, test_config, mapping)

In [8]:
%cd ../Yolov5_StrongSORT_OSNet
import sample_frame_tracker
# full_img_detection = sample_frame_tracker.run(test_file_path, save_vid=True, detect_only=True)
# full_img_detection

[Errno 2] No such file or directory: '../Yolov5_StrongSORT_OSNet'
/data/chanwutk/code/apperception/optimization_playground


ModuleNotFoundError: No module named 'sample_frame_tracker'

In [9]:
def display_detection(test_file_path, full_img_detection):
    import cv2
    test_frame = cv2.imread(test_file_path)
    for obj_idx, detection in full_img_detection.items():
        obj_cls, bbox = detection
        if obj_cls == 'car':
            x,y,w,h = list(map(int,bbox))
            cv2.rectangle(test_frame,(x-w//2,y-h//2),(x+w//2,y+h//2),(0,255,0),2)
            cv2.putText(test_frame, '_'.join([obj_cls, str(obj_idx)]), (x+w//2+5,y+h//2+5),0,0.3,(0,255,0))
    cv2.imshow('detection', test_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
from detection_estimation.utils import *
import matplotlib.pyplot as plt

In [11]:
# ego car trajectory
def prepare_ego(test_video):
    ego_trajectory = get_ego_trajectory(test_video)
    video_trajectory = fetch_camera_trajectory(test_video, database)
    sorted_ego_configs = [fetch_camera_config(e['fileName'], database) for e in video_trajectory]
    return sorted_ego_configs, ego_trajectory

In [9]:
# ego_speed = get_ego_speed(ego_trajectory)

In [10]:
# [time_to_nearest_frame(test_video, point.timestamp) for point in ego_trajectory]

In [11]:
import datetime
# test_timestamp = datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc)
# timestamp_to_nearest_trajectory(ego_trajectory, test_timestamp)

In [12]:
# test_point = [1772, 865, 0.0]
# point_to_nearest_trajectory(test_point, ego_trajectory)

In [13]:
from detection_estimation.sample_plan_algorithms import *

In [14]:
base_dir = '/home/yongming/workspace/research/apperception_new_local/boston-seaport'
test_img_base_dir = '/home/yongming/workspace/research/apperception/v1.0-mini/'

In [15]:
# import pickle
# with open(os.path.join(base_dir, f'frames.pickle'), "rb") as f:
#     df_sample_data = pickle.loads(f.read())
# df_sample_data
# i = 0
# for frame in df_sample_data['scene-0655-CAM_FRONT']['frames']:
#     if frame[2] == 1194:
#         print(i)
#         break
#     i += 1

In [16]:
from detection_estimation.detection_estimation import *
# ### Integration
# target_config_idx = 218
# video = 'scene-0655-CAM_FRONT'
# configs = df_sample_data[video]
# sorted_ego_config = [dict(zip(configs['columns'], frame))
#                      for frame in configs['frames']]
# len(sorted_ego_config)
# # all_car_loc3d = 

In [17]:
# current_ego_config = sorted_ego_config[target_config_idx]

In [18]:
car_loc3d_ground_truth = [(1991, 874), (1949.181, 873.164)]

In [19]:
from shapely.geometry import Point, Polygon
def generate_sample_plan_once(video, ego_config, mapping, next_frame_num, car_loc3d=None, target_car_detection=None, all_detection_info=None):
    if all_detection_info is None:
        assert target_car_detection and car_loc3d
        x,y,w,h = list(map(int, target_car_detection))
        car_loc2d = (x, y+h//2)
        car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
        car_bbox3d = None
        all_detections = []
        all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
        all_detection_info = construct_all_detection_info(
            current_frame, cam_segment_mapping, ego_trajectory, ego_config, all_detections)
    if all_detection_info:
        print(all_detection_info[0].road_type)
    next_sample_plan = generate_sample_plan(video, next_frame_num, all_detection_info,  50)
    next_frame = None
    if next_sample_plan.get_next_sample_frame_info():
        next_sample_frame_name, next_sample_frame_num, next_sample_frame_time = (
            next_sample_plan.get_next_sample_frame_info())
        print("next frame name", next_sample_frame_name)
        print("next frame num", next_sample_frame_num)
    #     print(next_sample_plan.action)
        next_frame = cv2.imread(test_img_base_dir+next_sample_frame_name)
#         cv2.imshow("next_frame", next_frame)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
    return next_sample_plan, next_frame


In [20]:
def construct_estimated_all_detection_info(current_frame, cam_segment_mapping, ego_config, ego_trajectory):
    all_detections = []
    full_img_detection = sample_frame_tracker.run(current_frame, save_vid=True, detect_only=True)
#     display_detection(current_frame, full_img_detection)
    for obj_idx, detection in full_img_detection.items():
        obj_cls, bbox = detection
        x,y,w,h = list(map(int,bbox))
        car_loc2d = (x,y+h//2)
#         print(car_loc2d)
        car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
        car_bbox3d = None
        estimate_3d = detection_to_img_segment(car_loc2d, cam_segment_mapping)
        if estimate_3d and estimate_3d.road_segment_info.segment_type in ['lane', 'laneSection']:
            car_loc3d = tuple(Polygon(estimate_3d.road_segment_info.segment_polygon).centroid.coords)
#             print(tuple(car_loc3d))
            all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
    print("all_detections", all_detections)
    all_detection_info = construct_all_detection_info(
        current_frame, cam_segment_mapping, ego_trajectory, ego_config, all_detections)
    return all_detection_info

In [21]:
def dry_run(sorted_ego_configs, start_frame_num, ego_trajectory, video):
    skipped_frame_num = []
    next_frame_num = start_frame_num
    action_type_counts = {}
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    display_video = cv2.VideoWriter(f'sampled_frames_{video.replace("/", "_")}.avi',fourcc, 10, (1600, 900))
    start_time = time.time()
    total_detection_time = 0
    total_sample_plan_time = 0
    for i in range(len(sorted_ego_configs)-1):
        current_ego_config = sorted_ego_configs[i]
        if current_ego_config['frameNum'] != next_frame_num:
            skipped_frame_num.append(current_ego_config['frameNum'])
            continue
        next_frame_num = sorted_ego_configs[i+1]['frameNum']
        cam_segment_mapping = map_imgsegment_roadsegment(current_ego_config)
        print("mapping length", len(cam_segment_mapping))
        current_frame = test_img_base_dir + current_ego_config['fileName']
        display_video.write(cv2.imread(current_frame))
        start_detection_time = time.time()
        all_detection_info = construct_estimated_all_detection_info(
            current_frame, cam_segment_mapping, current_ego_config, ego_trajectory)
        total_detection_time += time.time()-start_detection_time
        start_generate_sample_plan = time.time()
        next_sample_plan, next_frame = generate_sample_plan_once(
            video, current_ego_config, cam_segment_mapping, next_frame_num, all_detection_info=all_detection_info)
        total_sample_plan_time += time.time() - start_generate_sample_plan
        next_action_type = next_sample_plan.get_action_type()
        if next_action_type not in action_type_counts:
            action_type_counts[next_action_type] = 1
        else:
            action_type_counts[next_action_type] += 1
        next_frame_num = next_sample_plan.get_next_frame_num(next_frame_num)
    display_video.release()
    print("sorted_ego_config_length", len(sorted_ego_configs))
    print("number of skipped", len(skipped_frame_num))
    print(skipped_frame_num)
    print(action_type_counts)
    total_run_time = time.time()-start_time
    num_runs = len(sorted_ego_configs) - len(skipped_frame_num)
    print("total_run_time", total_run_time)
    print("avg run time", total_run_time/num_runs)
    print("total_detection_time", total_detection_time)
    print("avg detection time", total_detection_time/num_runs)
    print("total_generate_sample_plan_time", total_sample_plan_time)
    print("avg generate_sample_plan time", total_sample_plan_time/num_runs)

In [22]:
test_video1 = 'CAM_FRONT/n008-2018-08-27'
sorted_ego_configs1, ego_trajectory1 = prepare_ego(test_video1)
dry_run(sorted_ego_configs1, 2, ego_trajectory1, test_video1)


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.2646017074584961
mapping length 43
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((1810.3982084033205, 860.0461031162465),), car_loc2d=(1058, 532), car_bbox3d=None, car_bbox2d=(970, 466, 1146, 532)), obj_detection(id='car_1', car_loc3d=((1758.9680235211245, 866.051795447452),), car_loc2d=(826, 492), car_bbox3d=None, car_bbox2d=(810, 474, 842, 492)), obj_detection(id='car_1', car_loc3d=((1884.6902160362843, 872.6426751594242),), car_loc2d=(786, 493), car_bbox3d=None, car_bbox2d=(769, 475, 803, 493))]
lane
next frame name sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092262404.jpg
next frame num 27


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.3152017593383789
mapping length 43
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((1810.3982084033205, 860.0461031162465),), car_loc2d=(1079, 533), car_bbox3d=None, car_bbox2d=(999, 469, 1159, 533)), obj_detection(id='car_1', car_loc3d=((1884.6902160362843, 872.6426751594242),), car_loc2d=(785, 493), car_bbox3d=None, car_bbox2d=(770, 475, 800, 493)), obj_detection(id='car_1', car_loc3d=((1758.9680235211245, 866.051795447452),), car_loc2d=(830, 493), car_bbox3d=None, car_bbox2d=(813, 473, 847, 493))]
lane
next frame name sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092412404.jpg
next frame num 51


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


...

total mapping time:  0.7872545719146729
mapping length 58
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections []


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.8464281558990479
mapping length 58
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections []


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.6866118907928467
mapping length 58
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections []
sorted_ego_config_length 237
number of skipped 180
[13, 41, 66, 81, 90, 105, 118, 143, 160, 168, 185, 199, 210, 224, 263, 278, 289, 303, 328, 342, 367, 384, 408, 421, 446, 477, 488, 502, 515, 527, 541, 557, 582, 596, 607, 636, 645, 660, 674, 685, 698, 713, 722, 737, 752, 883, 896, 912, 921, 937, 952, 962, 977, 990, 1002, 1015, 1032, 1042, 1057, 1071, 1082, 1096, 1121, 1137, 1150, 1161, 1174, 1189, 1198, 1214, 1229, 1239, 1253, 1266, 1277, 1290, 1306, 1315, 1331, 1370, 1386, 1395, 1411, 1425, 1435, 1474, 1487, 1502, 1514, 1527, 1542, 1551, 1582, 1591, 1607, 1620, 1631, 1644, 1658, 1684, 1698, 1709, 1723, 1738, 1763, 1778, 1789, 1803, 1818, 1829, 1859, 1868, 1884, 1898, 1909, 

In [23]:
test_video2 = 'CAM_FRONT/n008-2018-08-01'
sorted_ego_configs2, ego_trajectory2 = prepare_ego(test_video2)
dry_run(sorted_ego_configs2, 2, ego_trajectory2, test_video2)


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.4129672050476074
mapping length 60
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((615.4809011304164, 1637.3069001205674),), car_loc2d=(1036, 531), car_bbox3d=None, car_bbox2d=(999, 483, 1073, 531)), obj_detection(id='car_1', car_loc3d=((548.168727975707, 1674.931277335449),), car_loc2d=(827, 526), car_bbox3d=None, car_bbox2d=(802, 484, 852, 526)), obj_detection(id='car_1', car_loc3d=((548.168727975707, 1674.931277335449),), car_loc2d=(885, 528), car_bbox3d=None, car_bbox2d=(853, 480, 917, 528)), obj_detection(id='car_1', car_loc3d=((618.8886816744962, 1640.5713210085441),), car_loc2d=(599, 554), car_bbox3d=None, car_bbox2d=(586, 482, 612, 554)), obj_detection(id='car_1', car_loc3d=((618.8886816744962, 1640.5713210085441

/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.3781287670135498
mapping length 58
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((619.4950977561506, 1639.892146379212),), car_loc2d=(540, 565), car_bbox3d=None, car_bbox2d=(527, 479, 553, 565)), obj_detection(id='car_1', car_loc3d=((548.168727975707, 1674.931277335449),), car_loc2d=(832, 539), car_bbox3d=None, car_bbox2d=(775, 479, 889, 539)), obj_detection(id='car_1', car_loc3d=((645.48790554265, 1618.5684959928494),), car_loc2d=(889, 496), car_bbox3d=None, car_bbox2d=(871, 472, 907, 496)), obj_detection(id='car_1', car_loc3d=((642.4554891057132, 1615.0472191851095),), car_loc2d=(962, 502), car_bbox3d=None, car_bbox2d=(944, 472, 980, 502))]
lane
next frame name samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRON

/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


...

/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary
/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:85: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for intersect in intersection:


total mapping time:  0.2899599075317383
mapping length 43
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((687.5019146385094, 1576.3385317678726),), car_loc2d=(878, 520), car_bbox3d=None, car_bbox2d=(851, 478, 905, 520)), obj_detection(id='car_1', car_loc3d=((744.777968055767, 1519.4513720645464),), car_loc2d=(976, 520), car_bbox3d=None, car_bbox2d=(951, 476, 1001, 520))]
lane


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary
/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:85: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for intersect in intersection:


total mapping time:  0.28183746337890625
mapping length 43
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((744.777968055767, 1519.4513720645464),), car_loc2d=(954, 515), car_bbox3d=None, car_bbox2d=(931, 477, 977, 515)), obj_detection(id='car_1', car_loc3d=((687.5019146385094, 1576.3385317678726),), car_loc2d=(880, 521), car_bbox3d=None, car_bbox2d=(853, 479, 907, 521)), obj_detection(id='car_1', car_loc3d=((730.7364787561393, 1556.0959890031118),), car_loc2d=(493, 552), car_bbox3d=None, car_bbox2d=(457, 506, 529, 552)), obj_detection(id='car_1', car_loc3d=((744.777968055767, 1519.4513720645464),), car_loc2d=(972, 522), car_bbox3d=None, car_bbox2d=(947, 476, 997, 522))]
lane
sorted_ego_config_length 229
number of skipped 158
[18, 27, 43, 57, 6

/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary
